In [1]:
debugging = True
IPTS = 17685

# Description 

Load a stack of images, select a region and then the average counts of the region vs the stack (index, TOF or lambda) will be displayed
Then this average counts will be compared to the theory signal of a given set of layers.

# Notebook Initialization 

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
%gui qt

In [4]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_resonance_imaging_experiment_vs_theory.ui')
o_builder = UiBuilder(ui_name = 'ui_resonance_imaging_layers_input.ui')

from __code import file_handler, utilities, fileselector
from __code.display_counts_of_region_vs_stack import ImageWindow
from NeuNorm.normalization import Normalization
from ImagingReso.resonance import Resonance

if debugging:
    ipts = IPTS
else:
    ipts = utilities.get_ipts()
working_dir = utilities.get_working_dir(ipts=ipts, debugging=debugging)
print("Working dir: {}".format(working_dir))

Working dir: /Volumes/my_book_thunderbolt_duo/IPTS/IPTS_17685


# Define Sample (temporaly location) 

In [5]:
import numpy as np
import os
import numbers

try:
    from PyQt4.QtGui import QFileDialog
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QMainWindow
except ImportError:
    from PyQt5.QtWidgets import QFileDialog
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QApplication

from __code.ui_resonance_imaging_layers_input import Ui_MainWindow as UiSampleMainWindow

In [8]:
class SampleWindow(QMainWindow):
    
    debugging = False
    stack = {} # used to initialize ImagingReso
    
    def __init__(self, parent=None, debugging=False):
        QMainWindow.__init__(self, parent=parent)
        self.ui = UiSampleMainWindow()
        self.ui.setupUi(self)
        self.setWindowTitle("Define Sample Layers")
    
        self.debugging = debugging
        
        self.initialize_ui()
        self.__debugging()
        
    def __debugging(self):
        '''Fill the table for debugging only!'''
        if not debugging:
            return
        
        _debug_table = [['CoAg','Co,Ag','1,1','20','0.9'],
                        ['Uranium','U','1','10','0.5']]
        for _row_index,_row in enumerate(_debug_table):
            for _col_index, _entry in enumerate(_row):
                _item = QtGui.QTableWidgetItem(_entry)
                self.ui.layer_table.setItem(_row_index, _col_index, _item)
        
    def initialize_ui(self):
        self.ui.check_groupBox.setVisible(False)
        
        _column_width = [300, 100, 100, 100, 100]
        for _index, _width in enumerate(_column_width):
            self.ui.layer_table.setColumnWidth(_index, _width)
            self.ui.example_table.setColumnWidth(_index, _width)
            
        _column_width = [80, 90, 90, 100]
        for _index, _width in enumerate(_column_width):
            self.ui.element_table.setColumnWidth(_index, _width)
        
    def validate_table_input_clicked(self):
        # collect table input
        nbr_row = self.ui.layer_table.rowCount()
        _table_dictionary = {}
        for _row_index in range(nbr_row):
            _dict = {}
            _layer_name = self.get_table_item(_row_index, 0)
            if _layer_name == '':
                break
            _dict['elements'] = self.format_string_to_array(string=self.get_table_item(_row_index, 1), data_type='str')
            _dict['stoichiometric_ratio'] = self.format_string_to_array(string=self.get_table_item(_row_index, 2), data_type='float')
            _dict['thickness'] = {'value': float(self.get_table_item(_row_index, 3)),
                                  'units': 'mm'}
            _dict['density'] = {'value': float(self.get_table_item(_row_index, 4)),
                                'units': 'g/cm3'}
            _table_dictionary[_layer_name] = _dict
        self.stack = _table_dictionary
        o_reso = Resonance(stack=self.stack)
        self.o_reso = o_reso
        
        self.fill_check_groupBox()
        self.ui.check_groupBox.setVisible(True)        
        
    def format_string_to_array(self, string='', data_type='str'):
        _parsed_string = string.split(',')
        result = []
        for _entry in _parsed_string:
            if data_type == 'str':
                result.append(str(_entry))
            elif data_type == 'float':
                result.append(float(_entry))
            else:
                raise ValueError("data_type not supported!")
        return result
    
    def get_table_item(self, row, col):
        _item = self.ui.layer_table.item(row, col)
        if _item is None:
            return ''
        _text = _item.text().strip()
        return _text

    def fill_check_groupBox(self):
        # fill layer's name
        _stack = self.o_reso.stack
        layers_name = list(_stack.keys())
        self.ui.layer_name_combobox.clear()
        self.ui.layer_name_combobox.addItems(layers_name)
        
    def element_combobox_clicked(self, element_selected):
        layer_selected = self.ui.layer_name_combobox.currentText()
        if layer_selected == '':
            return
        if element_selected == '':
            return
        
        _entry = self.stack[layer_selected][element_selected]
        number_of_atoms = float(self.stack[layer_selected]['atoms_per_cm3'][element_selected])
        self.ui.element_number_of_atoms.setText("{:6.3e}".format(number_of_atoms))
        density = str(self.stack[layer_selected][element_selected]['density']['value'])
        self.ui.element_density.setText(density)
        molar_mass = str(self.stack[layer_selected][element_selected]['molar_mass']['value'])
        self.ui.element_molar_mass.setText(molar_mass)
        
        self.fill_isotopes_table(element_selected)
        
    def fill_isotopes_table(self, element_selected=''):
        self.clear_isotopes_table()
        layer_selected = self.ui.layer_name_combobox.currentText()
        element_selected = self.ui.element_name_combobox.currentText()
        _entry = self.stack[layer_selected][element_selected]['isotopes']
        list_iso = _entry['list']
        list_density = _entry['density']['value']
        list_iso_ratio = _entry['isotopic_ratio']
        list_mass = _entry['mass']['value']
        
        nbr_iso = len(list_iso)
        for _row in range(nbr_iso):
            self.ui.element_table.insertRow(_row)

            # iso name
            _item = QtGui.QTableWidgetItem(list_iso[_row])
            _item.setFlags(QtCore.Qt.NoItemFlags)
            self.ui.element_table.setItem(_row, 0, _item)

            # density
            _item = QtGui.QTableWidgetItem("{:6.3e}".format(list_density[_row]))
            self.ui.element_table.setItem(_row, 1, _item)
            
            # iso. ratio
            _item = QtGui.QTableWidgetItem("{:6.3e}".format(list_iso_ratio[_row]))
            self.ui.element_table.setItem(_row, 2, _item)
            
            # molar mass
            _item = QtGui.QTableWidgetItem("{:6.3e}".format(list_mass[_row]))
            _item.setFlags(QtCore.Qt.NoItemFlags)
            self.ui.element_table.setItem(_row, 3, _item)

        self.calculate_sum_iso_ratio()
            
    def clear_isotopes_table(self):
        nbr_row = self.ui.element_table.rowCount()
        for _row in range(nbr_row):
            self.ui.element_table.removeRow(0)

    def calculate_sum_iso_ratio(self):
        nbr_row = self.ui.element_table.rowCount()
        total_iso_ratio = 0
        for _row in range(nbr_row):
            _iso_ratio = float(self.ui.element_table.item(_row, 2).text())
            total_iso_ratio += _iso_ratio
        self.ui.total_iso_ratio.setText("{:.2f}".format(total_iso_ratio))
            
    def layer_combobox_clicked(self, layer_selected):
        list_elements = self.stack[layer_selected]['elements']
        
        # fill list of elements for this layer
        self.ui.element_name_combobox.clear()
        self.ui.element_name_combobox.addItems(list_elements)
        
        # fill info for this layer
        _layer = self.stack[layer_selected]
        thickness = str(_layer['thickness']['value'])
        self.ui.layer_thickness.setText(thickness)
        density = str(_layer['density']['value'])
        self.ui.layer_density.setText(density)
        
    def element_table_edited(self, row, col):
        if col == 2:
            self.calculate_sum_iso_ratio()

_sample = SampleWindow(parent=None, debugging=debugging)
_sample.show()

# Select Stack Folder

In [6]:
input_folder_ui = fileselector.FileSelectorPanel(instruction='Select Input Folder', type='directory', start_dir=working_dir, multiple=False)
input_folder_ui.show()

# Load Stack

In [7]:
working_folder = input_folder_ui.result_full_path()
o_norm = Normalization()
o_norm.load(folder=working_folder, notebook=True)

# Define Sample 

# Select Region and display Counts vs file_index (or time)

In [8]:
_image = ImageWindow(
    stack=(o_norm.data['sample']['data']), working_folder=working_folder)
_image.show()